Simulation of matchgate circuits with matchgate noise:

In [6]:
from matchgates.generators import XY_circuit

n_qubits = 20  # number of qubits
dt = 0.1  # trotter step time interval
J = 0.5  # coupling strength
h = 0.23  # tranverse field
trotter_steps = 10  # number of trotter steps

# generate the XY and Ising circuits.
xy = XY_circuit(n_qubits=n_qubits, dt=dt, J=J, h=h, trotter_steps=trotter_steps)

Prepare initial state and observables:

In [7]:
from matchgates import Observable, ProductState

obsz = Observable(name="Z", qubits=[8], n_qubits=n_qubits)
obsyx = Observable(name="YX", qubits=[9, 10], n_qubits=n_qubits)

# prepare initial product state.
initial_state = ProductState.neel(even=True, n_qubits=n_qubits)

To add noise to the circuit, need to make a choice of noise for each gate in the circuit.
If all two qubit gates in the circuit have the same noise model, this noise model
can be set up in a simple way:

In [8]:
from matchgates import SingleGateNoise

p = 0.002
noise_channel = SingleGateNoise.matchgate_depolarizing(p=p)
# matchgate depolarizing is equal probability of perturbation by
# any matchgate Pauli (Z, ZZ, XY, YX, YY, XX).

noisy_xy = xy.add_two_qubit_uniform_noise(noise_channel)  # Add noise to circuit.

A noisy circuit can be simulated using the simulate_noisy method.
As the simulation is done by monte-carlo it is necessary to repeat
over some number of repetitions, and this can be distributed over 
multiple jobs. What will be returned is a single dictionary with the 
mean values for all observables in the list, averaged over the
repetitions specified.

In [9]:
n_jobs = 20
repetitions = 1000
results = noisy_xy.simulate_noisy(n_jobs, [obsz, obsyx], repetitions, initial_state)
print("Results for XY circuit with noise:", results)

Results for XY circuit with noise: {'Z(8,)': np.float64(0.3796567285413587), 'YX(9, 10)': np.float64(0.11081567138737675)}


In practice, I find it most convenient to loop the above over a large number of iterations
and after each iterations append the results to a csv file. Then it is possible to access the
data over multiple iterations and compute statistics (mean, std, etc.) to check how well 
converged the results are.

In [11]:
from time import time
import pandas as pd

start_time = time()

iterations = 4

for i in range(iterations):
    results = noisy_xy.simulate_noisy(n_jobs, [obsz, obsyx], repetitions, initial_state)
    new_results = {}
    for key in results.keys():
        new_results[key] = [results[key]]
    df = pd.DataFrame(new_results)
    if i == 0:
        df.to_csv(
            f"examples/XY_p{p}_N{n_qubits}_J{J}_h{h}_steps{trotter_steps}_rep{repetitions}.csv",
            mode="w",
            header=True,
            index=False,
        )
    else:
        df.to_csv(
            f"examples/XY_p{p}_N{n_qubits}_J{J}_h{h}_steps{trotter_steps}_rep{repetitions}.csv",
            mode="a",
            header=False,
            index=False,
        )
    print("Iteration: ", i)
    print("Time taken is ", time() - start_time)
    start_time = time()

Iteration:  0
Time taken is  12.995606422424316
Iteration:  1
Time taken is  12.691557884216309
Iteration:  2
Time taken is  13.054940223693848
Iteration:  3
Time taken is  12.921587705612183
